# Train

Following from [Preprocessing](https://github.com/TheNerdyCat/deepfake-detection-challenge/blob/master/output/preprocessing.ipynb), this stage will look at data augmentation and subsequently training the model.

We will read our extracted faces using OpenCV and perform any data augmentation. Following this, we will define X and X_val. Then we'll read the metadata to label the extracted faces as FAKE or REAL, defining them into y and y_val.

After we have our training data and validation data ready and shuffled, we'll train our model.

In [8]:
import pandas as pd
import numpy as np

import os
import json # To read the metadata

import keras
from keras import Model, Sequential
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import LearningRateScheduler

import torch
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from tqdm.notebook import tqdm
import random

import warnings
warnings.filterwarnings("ignore")

In [26]:
train_sample_images_path = '../input/train_images/'
train_sample_images = os.listdir(train_sample_images_path)

# Read in metadata
with open('../input/train_metadata/metadata0.json') as json_file:
    metadata = json.load(json_file)

X = []
for img in train_sample_images:
    img = train_sample_images_path + img
    img = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)
    X.append(img)

y = []
for label in train_sample_images:
    if metadata[label.split('_')[0] + '.mp4']['label'] == 'REAL':
        y.append(0)
    else:
        y.append(1)

### Undersampling to go here

In [27]:
def shuffle(X, y):
    new_train = []
    for m, n in zip(X, y):
        new_train.append([m, n])
    random.shuffle(new_train)
    X, y = [], []
    for x in new_train:
        X.append(x[0])
        y.append(x[1])
    return X, y

In [30]:
X, y = shuffle(X, y)

# Create X_val from 10% of X
X_val = X[:round(len(X) / 100 * 25)]
X = X[round(len(X) / 100 * 25):]

# Create y_val from 10% of y
y_val = y[:round(len(y) / 100 * 25)]
y = y[round(len(y) / 100 * 25):]

In [31]:
X, y = shuffle(X, y)
X_val, y_val = shuffle(X_val, y_val)

In [ ]:
def InceptionLayer(a, b, c, d):
    def func(x):
        x1 = Conv2D(a, (1, 1), padding='same', activation='elu')(x)
        
        x2 = Conv2D(b, (1, 1), padding='same', activation='elu')(x)
        x2 = Conv2D(b, (3, 3), padding='same', activation='elu')(x2)
            
        x3 = Conv2D(c, (1, 1), padding='same', activation='elu')(x)
        x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides=1, padding='same', activation='elu')(x3)
        
        x4 = Conv2D(d, (1, 1), padding='same', activation='elu')(x)
        x4 = Conv2D(d, (3, 3), dilation_rate=3, strides=1, padding='same', activation='elu')(x4)
        y = Concatenate(axis = -1)([x1, x2, x3, x4])
            
        return y
    return func
    
def define_model(shape=(256, 256, 3)):
    x = Input(shape=shape)
    
    x1 = InceptionLayer(1, 4, 4, 2)(x)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
    
    x2 = InceptionLayer(2, 4, 4, 2)(x1)
    x2 = BatchNormalization()(x2)        
    x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)        
        
    x3 = Conv2D(16, (5, 5), padding='same', activation='elu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
    x4 = Conv2D(16, (5, 5), padding='same', activation='elu')(x3)
    x4 = BatchNormalization()(x4)
    x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
    
    y = Flatten()(x4)
    y = Dropout(0.5)(y)
    y = Dense(16)(y)
    y = LeakyReLU(alpha=0.1)(y)
    y = Dropout(0.5)(y)
    y = Dense(1, activation='sigmoid')(y)
    model = Model(inputs=x, outputs=y)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-4))
    return model

df_model = define_model()

In [ ]:
lrs = [1e-3, 5e-4, 1e-4]
def schedule(epoch):
    return lrs[epoch]

In [ ]:
kfolds = 5
losses = []

models = []
i = 0
while len(models) < kfolds:
    model = define_model()
    model.fit([X], [y], epochs=2, callbacks=[LearningRateScheduler(schedule)])
    pred = model.predict([X_val])
    loss = log_loss(y_val, pred)
    losses.append(loss)
    print('Fold ' + str(i) + ' model loss: ' + str(loss))
    if loss < 0.68:
        models.append(model)
    else:
        print('##############')
        print('Retraining')
        print('##############')
    K.clear_session()
    del model
    gc.collect()
    i += 1